# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [1]:
# import libraries
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from time import time
from functools import reduce

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import avg, col, concat, desc, explode, lit, min, max, split, udf, isnan, when, rank, from_unixtime
from pyspark.sql.types import IntegerType, DateType
from pyspark.sql.window import Window

from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import StandardScaler, VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.feature import MinMaxScaler
from pyspark.mllib.evaluation import MulticlassMetrics

%matplotlib inline

In [2]:
# create a Spark session
spark = SparkSession \
    .builder \
    .appName("Sparkify") \
    .getOrCreate()

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [3]:
file_path = 'mini_sparkify_event_data.json'
df = spark.read.json(file_path)
df.persist()

DataFrame[artist: string, auth: string, firstName: string, gender: string, itemInSession: bigint, lastName: string, length: double, level: string, location: string, method: string, page: string, registration: bigint, sessionId: bigint, song: string, status: bigint, ts: bigint, userAgent: string, userId: string]

In [4]:
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [5]:
df.count() #number of rows

286500

In [6]:
len(df.columns) #number of columns

18

In [7]:
df.describe().show()

+-------+------------------+----------+---------+------+------------------+--------+-----------------+------+-----------------+------+-------+--------------------+-----------------+--------------------+------------------+--------------------+--------------------+-----------------+
|summary|            artist|      auth|firstName|gender|     itemInSession|lastName|           length| level|         location|method|   page|        registration|        sessionId|                song|            status|                  ts|           userAgent|           userId|
+-------+------------------+----------+---------+------+------------------+--------+-----------------+------+-----------------+------+-------+--------------------+-----------------+--------------------+------------------+--------------------+--------------------+-----------------+
|  count|            228108|    286500|   278154|278154|            286500|  278154|           228108|286500|           278154|286500| 286500|            

In [8]:
#missing values
def get_nulls_count(df):
    """
    Checks for the null values for each column and returns the count of null values.
    
    Arguments :
    df : spark dataframe
    
    Returns:
    None. print mising values count for each column
    """
    print('Missing Values\n')
    
    for col in df.columns:
        null_count = df.filter((isnan(df[col])) | (df[col].isNull()) | (df[col] == '')).count()
        if null_count > 0:
            print(f"{col} : {null_count}")
            
get_nulls_count(df)

Missing Values

artist : 58392
firstName : 8346
gender : 8346
lastName : 8346
length : 58392
location : 8346
registration : 8346
song : 58392
userAgent : 8346
userId : 8346


There are so many null counts equal to 8346 so it is obvious there is a pattern. Let's look at the userId And some have 58392 null value counts which are related to song info such as song title, artist, length of the song

In [9]:
df.select("userId").dropDuplicates().sort("userId").show(5)

+------+
|userId|
+------+
|      |
|    10|
|   100|
|100001|
|100002|
+------+
only showing top 5 rows



User ids have empty strings let's see those user behavior to get the sense of who are those users

Which page did user id "" (empty string) visit?

In [10]:
df.filter(df.userId == '') \
    .select(col('page') \
    .alias('blank_pages')) \
    .dropDuplicates().show()

+-------------------+
|        blank_pages|
+-------------------+
|               Home|
|              About|
|Submit Registration|
|              Login|
|           Register|
|               Help|
|              Error|
+-------------------+



Which page did user id "" (empty string) NOT visit?

In [11]:
# filter for users with blank user id
blank_pages = df.filter(df.userId == '') \
    .select(col('page') \
    .alias('blank_pages')) \
    .dropDuplicates()

# get a list of possible pages that could be visited
all_pages = df.select('page').dropDuplicates()

# find values in all_pages that are not in blank_pages
# these are the pages that the blank user did not go to
for row in set(all_pages.collect()) - set(blank_pages.collect()):
    print(row.page)

Roll Advert
Settings
Logout
Cancel
Add Friend
NextSong
Cancellation Confirmation
Thumbs Down
Save Settings
Downgrade
Submit Upgrade
Add to Playlist
Thumbs Up
Submit Downgrade
Upgrade


Perhaps it represents users who have not signed up yet or who are signed out and are about to log in. So we can remove these users

In [12]:
df.select('userId').distinct().count() #Unique Users

226

In [13]:
df_clean = df.dropna(how = "any", subset = ["userId", "sessionId"])
df_clean = df_clean.filter(df["userId"] != "")

In [14]:
df_clean.select('userId').distinct().count() #Unique Users after cleaning

225

In [15]:
df_clean.count() #number of rows after cleaning

278154

Let's look at the data type issues and column value issues

In [16]:
df_clean.persist()

DataFrame[artist: string, auth: string, firstName: string, gender: string, itemInSession: bigint, lastName: string, length: double, level: string, location: string, method: string, page: string, registration: bigint, sessionId: bigint, song: string, status: bigint, ts: bigint, userAgent: string, userId: string]

Registration and ts(time stamp) are in bigint format instead of date format. Let's transform these two columns

In [17]:
df_clean.select('registration','ts').show(5)

+-------------+-------------+
| registration|           ts|
+-------------+-------------+
|1538173362000|1538352117000|
|1538331630000|1538352180000|
|1538173362000|1538352394000|
|1538331630000|1538352416000|
|1538173362000|1538352676000|
+-------------+-------------+
only showing top 5 rows



In [18]:
#transforming ts and registration
get_date = udf(lambda x : datetime.datetime.fromtimestamp(x / 1000.0).strftime("%Y-%m-%d %H:%M:%S"))
df_clean = df_clean.withColumn('eventTime' , get_date('ts'))
df_clean = df_clean.withColumn('registrationTime' , get_date('registration'))
df_clean.select('registrationTime','eventTime').show(5)

+-------------------+-------------------+
|   registrationTime|          eventTime|
+-------------------+-------------------+
|2018-09-28 22:22:42|2018-10-01 00:01:57|
|2018-09-30 18:20:30|2018-10-01 00:03:00|
|2018-09-28 22:22:42|2018-10-01 00:06:34|
|2018-09-30 18:20:30|2018-10-01 00:06:56|
|2018-09-28 22:22:42|2018-10-01 00:11:16|
+-------------------+-------------------+
only showing top 5 rows



In [ ]:
# get unique values of all columns / features to investigate the column values
for col in df_clean.columns:
    df_clean.select([col]).distinct().show()

+--------------------+
|              artist|
+--------------------+
|      The Black Keys|
|         Silverstein|
|           Kate Nash|
|        Yann Tiersen|
|    Jane's Addiction|
|          Tim Hughes|
|          Carl Craig|
|Dashboard Confess...|
|Yonder Mountain S...|
|           Los Lobos|
|Pete Rock & C.L. ...|
|        Ziggy Marley|
|      Jarabe De Palo|
|               Rufio|
|WC And The Maad C...|
|      Jorge Gonzalez|
|                Silk|
|  The Watts Prophets|
|            La Shica|
|        Generation X|
+--------------------+
only showing top 20 rows

+---------+
|     auth|
+---------+
|Cancelled|
|Logged In|
+---------+

+----------+
| firstName|
+----------+
|    Maddox|
|    Karter|
|     Lucas|
|     Grace|
|  Antonina|
|   Lorelei|
|   Adriana|
|  Isabella|
|     James|
|    Brooke|
|  Benjamin|
|  Giovanni|
|     Davis|
|    Nathan|
|   Antonio|
| Christian|
|     Brisa|
|Alexandria|
|   Spencer|
|    Jordan|
+----------+
only showing top 20 rows

+------+
|g

- Location has city name and state name we can extract the state name for better visualization.

In [ ]:
#extracting state
get_state = udf(lambda x : x[-2:])
df_clean = df_clean.withColumn('locationState' , get_state('location'))
df_clean.select('locationState').distinct().show(5)

# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

In [ ]:
#time range in the data
max_date = df_clean.select('eventTime').agg({'eventTime': 'max'}).collect()[0][0]
min_date = df_clean.select('eventTime').agg({'eventTime': 'min'}).collect()[0][0]
range_date = f'min : {min_date} , max : {max_date}'
print(range_date)

So, We have 2 months of data

In [ ]:
#distinct page events
df_clean.select('page').distinct().show()

In [ ]:
#Create feature churn
churn_func = udf(lambda x : 1 if x == 'Cancellation Confirmation' else 0, IntegerType())

df_clean = df_clean.withColumn('churn_event', churn_func('page'))
df_clean = df_clean.withColumn('churn', max('churn_event').over(Window.partitionBy('userId')))

In [ ]:
churn_count = df_clean.dropDuplicates(['userId', 'churn']).groupby(['churn']).count().toPandas()

sns.barplot(x='churn', y='count', data=churn_count);
plt.xlabel('churn')
plt.ylabel('count')
plt.title('How many user churned?');

### Distribution of registered days between churned and non churned users

In [ ]:
#plot the registered_days for churned and not churned users
registered_days = df_clean.select('userId','churn','registration','ts')
registered_days = registered_days.withColumn('registeredDays',(df_clean.ts - df_clean.registration)/(1000*3600*24))\
                  .groupBy('userId','churn').agg({'registeredDays': 'max'}).toPandas()

registered_days.rename(columns= {'max(registeredDays)' : 'registeredDays'},inplace = True)
registered_days.head()

In [ ]:
sns.boxplot(data = registered_days , x = 'churn', y = 'registeredDays')
plt.xlabel('Churn')
plt.ylabel('Registered days')
plt.title('Distribution of registered days of churned and non-churned users')

In [ ]:
print(registered_days.groupby('churn')['registeredDays'].median())

There is a difference between distribution of registered days between two groups. 50% of the churned users left the service within first 50 days

### How long do users listen to the songs on average in single session

In [ ]:
avg_session_length = df_clean.select('userId','churn','sessionId','length').groupBy('userId','churn','sessionId').agg({'length' : 'mean'})\
        .groupBy('userId','churn').agg({'avg(length)' : 'mean'}).toPandas()

avg_session_length.rename(columns= {'avg(avg(length))' : 'avgSessionLength'},inplace = True)
avg_session_length.head()

In [ ]:
sns.boxplot(data = avg_session_length , x = 'churn', y = 'avgSessionLength')
plt.xlabel('Churn')
plt.ylabel('Average session length')
plt.title('Distribution of Average session length of churned and non-churned users');

###  Is there any difference in the Location?

In [ ]:
# locations (state) of the users
location_state = df_clean.dropDuplicates(['userId', 'locationState']).groupby('locationState').agg({'churn' : 'mean'}).toPandas()
location_state.rename(columns= {'avg(churn)' : 'churnProportion'},inplace = True)
location_state.head()

In [ ]:
fig = plt.figure(figsize=(20, 6))

sns.barplot(x='locationState', y='churnProportion', data=location_state, color = sns.color_palette()[0]);
plt.xlabel('location (state)')
plt.ylabel('Churn Proportion')
plt.title('Churn proportion in each state');
plt.legend(title='churn', loc='best');

### Distribution of number of unique sessions for churn and non churn users

In [ ]:
unique_sessions = df_clean.select('userId', 'churn', 'sessionId').drop_duplicates().groupBy(['userId', 'churn'])\
                  .count().toPandas()
 
# compare two groups of users
g = sns.FacetGrid(unique_sessions, col="churn", sharey=False)
g.map(plt.hist, "count");

Number of unique sessions per user is less for churn users

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

In [ ]:
# gender of the user
gender_df = df_clean.select(['userId', 'gender']).dropDuplicates(['userId']).replace(['F', 'M'], ['0', '1'], 'gender')
gender_df = gender_df.withColumn('gender', gender_df.gender.cast('int'))
gender_df.show(5)

In [ ]:
# level of the user
payment_df = df_clean.select(['userId', 'level']).dropDuplicates(['userId']).replace(['paid', 'free'], ['0', '1'], 'level')
payment_df = payment_df.withColumn('level', payment_df.level.cast('int'))
payment_df.show(5)

In [ ]:
# did the user downgrade
downgrade_event = udf(lambda x: 1 if x == 'Submit Downgrade' else 0, IntegerType())
downgrade_df = df_clean.select(['userId','page'])
downgrade_df = downgrade_df.withColumn('downgrade', downgrade_event('page'))
downgrade_df = downgrade_df.withColumn('downgrade', max('downgrade').over(Window.partitionBy('userId')))
downgrade_df = downgrade_df.select(['userId','downgrade']).dropDuplicates(['userId']) 
downgrade_df = downgrade_df.withColumn('downgrade', downgrade_df.downgrade.cast('int'))
downgrade_df.show(5)

In [ ]:
# number of songs the user listened to in total
num_songs = df_clean.select('userID','song').groupBy('userID').count()
num_songs.show(5) 

In [ ]:
# number of Thumbs-Up/Down
thumbsup_event = udf(lambda x: 1 if x == 'Thumnbs Up' else 0, IntegerType())
df_clean = df_clean.withColumn('thumbsUp', thumbsup_event('page'))
num_thumbs_up = df_clean.select('userID','thumbsUp').groupBy('userID').mean().withColumnRenamed('count', 'num_thumbs_up') 
print(num_thumbs_up.show(5))
num_thumbs_down = df_clean.select('userID','page').where(df_clean.page == 'Thumbs Down').groupBy('userID').mean().withColumnRenamed('count', 'num_thumbs_down') 
print(num_thumbs_down.show(5))

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.